# SVC - C-Support Vector Classification

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Sequential Feature Selection* (**SFS**, ou Seleção Sequencial de Atributos em português) por remoção de atributo, a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

### Modelo

O algoritmo deste notebook é o Support Vector Machine (SVM) para Classificação (ou seja, SVC). O seu método de segregação de dados pelo corte por hiperplanos possui muitas possibilidades geométricas e de proporção, assim, quando há muitos atributos e não se sabe se apresentam algum padrão de distribuição multidimensional possível, saber o kernel utilizado que melhor se adeque pode ser um problema de tentativa e erro.

Os hiperparâmetros a serem otimizados são:
* `C`: coeficiente da relação entre a margem da transição do hiperplano e a quantidade de objetos não bem separados - um "C" menor aumenta a margem e pode abranger objetos em si, de forma que, ao custo de poder errar mais, generaliza-se melhor;
* `gamma`: define a influência individual de um objeto sobre o encaixe ("fitting") do hiperplano (para sigmoid, poly e RBF);
* `kernel`: método de traçagem do plano, utilizaremos os 4 citados;
* `coef0`: termo independente do kernel polinomial e do sigmoidal;
* `degree`: grau do kernel polinomial.

## Carregamento dos dados

In [1]:
from pandas import read_csv

# Dados de treino
X_treino = read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

In [2]:
# Parâmetros
SEMENTE_ALEATORIA = 9 # Random state
num_max_atributos = len(X_treino.columns) - 1

X_treino = X_treino.values

## Definição de funções para o optuna

O código abaixo define a função que cria uma instância do modelo. É nela que a pipeline é montado de acordo com as sugestões definidas pelo optuna.

In [3]:
from sklearn.svm import SVC # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import SequentialFeatureSelector # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "C": trial.suggest_float("C_coef", 0.001, 10),
        "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "gamma": trial.suggest_categorical("fit_hiperplano", ["scale", "auto"]),
        "coef0": trial.suggest_float("coef0", 0, 10),
        "degree": trial.suggest_int("grau_polinomial", 1, 5),
        "max_iter": -1,
        "class_weight": "balanced", # Dados desbalanceados
    }

    normalizacao = trial.suggest_categorical("normalizacao", ["standard", "minmax"])
    usar_pca = trial.suggest_categorical("PCA", [True, False])
    usar_sfs = trial.suggest_categorical("SFS", [True, False])

    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para RFE
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA
        
    
    steps = []

    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    else:
        steps.append(("normalizador", MinMaxScaler()))

    if usar_sfs:
        steps.append(("sfs", SequentialFeatureSelector(
            SVC(**parametros), 
            n_features_to_select=num_atributos,
            direction="backward",
            scoring=f2_score,
            cv=5,
            n_jobs=1,
        )))

    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", SVC(**parametros)))

    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

O código abaixo define a função que estima o desempenho do modelo de cada trial. Essa é a função que terá valor de retorno maximizado pelo optuna.

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import numpy as np

f2_score = make_scorer(fbeta_score, beta=2)

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna.

    Faz validação cruzada estratificada com métrica F2.

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "SVC_optuna_"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:39:36,547] A new study created in RDB with name: SVC_optuna_


In [ ]:
# Parâmetros
NUM_FOLDS = 10 # Validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:43:31,328] Trial 0 finished with value: 0.8011583175355405 and parameters: {'C_coef': 8.871948865278805, 'kernel': 'poly', 'fit_hiperplano': 'scale', 'coef0': 7.583752299522676, 'grau_polinomial': 5, 'normalizacao': 'minmax', 'PCA': True, 'SFS': False, 'num_atributos': 6, 'num_dimensoes': 5}. Best is trial 0 with value: 0.8011583175355405.
[I 2025-11-04 00:43:32,249] Trial 1 finished with value: 0.8379695636065071 and parameters: {'C_coef': 5.6486994755564375, 'kernel': 'poly', 'fit_hiperplano': 'scale', 'coef0': 1.0755358857473984, 'grau_polinomial': 3, 'normalizacao': 'minmax', 'PCA': False, 'SFS': False, 'num_atributos': 7, 'num_dimensoes': 3}. Best is trial 1 with value: 0.8379695636065071.
[I 2025-11-04 00:43:55,565] Trial 2 finished with value: 0.8374340157342633 and parameters: {'C_coef': 7.894650427962061, 'kernel': 'poly', 'fit_hiperplano': 'auto', 'coef0': 0.8794653726929291, 'grau_polinomial': 5, 'normalizacao': 'minmax', 'PCA': True, 'SFS': True, 'num_atrib

## DataFrame dos trials

O código abaixo mostra o DataFrame contruído pelo objeto de estudo do optuna contendo todos os parâmetros de cada trial.

In [ ]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

In [ ]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

[3] Página do Support Vector Machine no scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

[4] Página do Support Vector Machine em classificação (SVC) na documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

[5] Helmy, Bahaa; Aibin, Michal. *The C Parameter in Support Vector Machines*. Acesso em: 01/11/2025. Disponível em: https://www.baeldung.com/cs/ml-svm-c-parameter